In [1]:
from pprint import pprint
import jmespath
import requests
from bs4 import BeautifulSoup as BS4
from textwrap import dedent
import re

In [2]:
req = requests.get('https://www.selver.ee/')

In [3]:
soup = BS4(req.content, features='html.parser')

In [4]:
#esilehelt URL otsingu regex
url_class = re.compile(r'(item level0 nav.\w+.+\d)')
#Hinna ja toote otsingu regex
product_class = re.compile(r'(col-lg-3 \w+).+') 
product_price = re.compile(r'("price")')

In [5]:
#Leia kõik HTML Li tagilt mis hoiab URLi
find_tag = soup.find_all('li',{'class':url_class})

In [6]:
#leia URL'id ja lista toote nimekirja pikkus
get_urls = [i.find_next('a').get('href')+"/?limit=96&p=" for i in find_tag]

In [7]:
get_urls

['https://www.selver.ee/minu-tooted/?limit=96&p=',
 'https://www.selver.ee/uudistooted/?limit=96&p=',
 'https://www.selver.ee/soodushinnaga-tooted/?limit=96&p=',
 'https://www.selver.ee/eritooted/?limit=96&p=',
 'https://www.selver.ee/hulgipakkumised/?limit=96&p=',
 'https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=',
 'https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=',
 'https://www.selver.ee/piimatooted-munad-void/?limit=96&p=',
 'https://www.selver.ee/juustud/?limit=96&p=',
 'https://www.selver.ee/leivad-saiad-kondiitritooted/?limit=96&p=',
 'https://www.selver.ee/valmistoidud/?limit=96&p=',
 'https://www.selver.ee/kuivained-hoidised/?limit=96&p=',
 'https://www.selver.ee/maitseained-ja-puljongid/?limit=96&p=',
 'https://www.selver.ee/kastmed-olid/?limit=96&p=',
 'https://www.selver.ee/maiustused-kupsised-naksid/?limit=96&p=',
 'https://www.selver.ee/kulmutatud-toidukaubad/?limit=96&p=',
 'https://www.selver.ee/joogid/?limit=96&p=',
 'https://www.selver.ee/lastekaubad/?limi

In [11]:
#Kogu html linkide kogust
product_html = requests.get(get_urls[5])
#puhasta ilusa seebiga
soup = BS4(product_html.content, features='html.parser')
#leia toote ja hinna kategoorijad li tagist
find_product_group = soup.find_all('li',{'class':product_class})
#leia hinnad
find_product_group_prices = soup.find_all('span',{'class':'price'})
#võta linki pealkirjad ehk toote nimed
find_product_group_titles = [i.find_next('a').get('title') for i in find_product_group]
#leia kõik hinnad
prices = [dedent(i.text.replace(u'\xa0',' ')) for i in find_product_group_prices if len(i) > 1]

In [8]:
html_list = list()
dict_of_products = dict()

In [9]:
#start of the urls and end of the URLS
for z in range(5,23):
    #Find product last page
    product_html = requests.get(get_urls[z])
    #puhasta ilusa seebiga
    soup = BS4(product_html.content, features='html.parser')
    try:
        find_product_last_page = soup.find('a',{'class':'last'}).text
    except AttributeError:
        find_product_last_page = 3
    
    for i in range(1,int(find_product_last_page)+1):
        product_html = requests.get(get_urls[z]+str(i))
        soup = BS4(product_html.content, features='html.parser')
        html_list.append(soup)  
        print(f"Page URI {get_urls[z]+str(i)} Last page is {find_product_last_page}")

    for y in range(len(html_list)):  
        find_product_group = html_list[y].find_all('li',{'class':product_class})
        find_product_group_prices = html_list[y].find_all('span',{'class':'price'})
        find_product_group_titles = [i.find_next('a').get('title') for i in find_product_group]
        prices = [dedent(i.text.replace(u'\xa0',' ')) for i in find_product_group_prices if len(i) > 1]
        for t,f in zip(find_product_group_titles,prices):
            dict_of_products.update({t:f})

Page URI https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=1 Last page is 4
Page URI https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=2 Last page is 4
Page URI https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=3 Last page is 4
Page URI https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=4 Last page is 4
Page URI https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=1 Last page is 9
Page URI https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=2 Last page is 9
Page URI https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=3 Last page is 9
Page URI https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=4 Last page is 9
Page URI https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=5 Last page is 9
Page URI https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=6 Last page is 9
Page URI https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=7 Last page is 9
Page URI https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=8 Last page is 9
Page URI https://www.selver.ee/liha-ja-kalatooted/?l

Page URI https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=1 Last page is 12
Page URI https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=2 Last page is 12
Page URI https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=3 Last page is 12
Page URI https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=4 Last page is 12
Page URI https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=5 Last page is 12
Page URI https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=6 Last page is 12
Page URI https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=7 Last page is 12
Page URI https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=8 Last page is 12
Page URI https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=9 Last page is 12
Page URI https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=10 Last page is 12
Page URI https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=11 Last page is 12
Page URI https://www.selver.ee/majapidami

In [23]:
import pandas as pd

In [64]:
df = pd.DataFrame(columns=['Toode','Hind'])

In [66]:
df['Toode'] = dict_of_products.keys()
df['Hind'] = dict_of_products.values()

In [27]:
csv_file = pd.read_csv('selver_data.csv')

In [30]:
csv_file.drop(['Unnamed: 0'], axis=1, inplace=True)

In [35]:
choices = [i for i in csv_file['Toode']]

In [37]:
len(choices)

10462